#Fyp 30%

The purpose of this notebook is to do preprocessing of Liar dataset with merged labels that we saved in previous data analysis notebook

extract features


1.   sentiment
2.   linguistic (part of speech, name entity)
3.   bert sentence embedding





#Import Libraries and Packages

Absolutely, here's a brief explanation of each of the Python libraries used in your code:

1. **nltk (Natural Language Toolkit)**: it is used for tokenization, POS tagging, named entity recognition and stopwords.

4. **BertTokenizer**: A tokenizer specific to the BERT model (from the transformers library). It's used for converting text into tokens that correspond to BERT's vocabulary.

5. **tensorflow**: This is a free and open-source software library for machine learning and artificial intelligence. It's used here as the backend for the transformers library.

8. **TextBlob**: This is a Python library for processing textual data. used for sentiment analysis, it's used for sentiment analysis class.


In [ ]:
!pip install transformers
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install pandas
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.7 MB/s eta 0:00:00
2023-09-23 08:23:40.964887: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 08:23:42.453867: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-23 08:23:44.234689: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See 

In [ ]:
import pandas as pd
import os
import string
import spacy
import pickle
import tensorflow as tf
from textblob import TextBlob
from transformers import BertTokenizer, TFBertModel

#Prerequisite

mounting notebook with google

datahandler class to load and save dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

class DataHandler:
    def __init__(self, train_file, val_file, test_file):
        self.train_file = train_file
        self.val_file = val_file
        self.test_file = test_file

    def load_csv_data(self, delimiter=','):
        self.train_data = pd.read_csv(self.train_file, delimiter=delimiter)
        self.val_data = pd.read_csv(self.val_file, delimiter=delimiter)
        self.test_data = pd.read_csv(self.test_file, delimiter=delimiter)
        return self.train_data, self.val_data, self.test_data

    def save_csv_data(self, train_data, val_data, test_data, folder=None, delimiter=','):
        if folder:
            train_path = os.path.join(folder, 'train.csv')
            val_path = os.path.join(folder, 'val.csv')
            test_path = os.path.join(folder, 'test.csv')
        else:
            train_path = self.train_file
            val_path = self.val_file
            test_path = self.test_file

        train_data.to_csv(train_path, index=False, sep=delimiter)
        val_data.to_csv(val_path, index=False, sep=delimiter)
        test_data.to_csv(test_path, index=False, sep=delimiter)

base_path = '/content/drive/MyDrive'  # base directory where the "liar" folder is located
foldername = 'Liar collapsed labels'
train_path = os.path.join(base_path, foldername, 'train.csv')
val_path = os.path.join(base_path, foldername, 'val.csv')
test_path = os.path.join(base_path, foldername, 'test.csv')
data_handler = DataHandler(train_path, val_path, test_path)
liar_df = data_handler.load_csv_data()
liar_df[0].sample(n=2)



,Statement,Label,Label_Value
1056,When Tallahassee politicians and bureaucrats t...,True,1
4256,Household income in America has gone down for ...,False,0


#Define Text Preprocessor Class
This class will be used for text preprocessing tasks such as tokenization and stopword remova

In [ ]:

# Load spaCy NER and POS models
nlp = spacy.load("en_core_web_sm")

class Preprocessor:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def preprocess(self, text):
        # tokenize the text
        tokenized_text = self.tokenizer.tokenize(text)

        # remove punctuation
        tokenized_text = [token for token in tokenized_text if token not in string.punctuation]

        return tokenized_text

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
preprocessor = Preprocessor(tokenizer)


#POS Tagger and Named Entity Recognizer Classes

In [ ]:

class NamedEntityRecognizer:
    def recognize(self, text):
        doc = nlp(text)
        return doc

class POSTagger:
    def tag(self, text):
        doc = nlp(text)
        return [(token.text, token.pos_) for token in doc]

class SentimentAnalyzer:
    def analyze(self, text):
        sentiment = TextBlob(text).sentiment
        return sentiment.polarity, sentiment.subjectivity

sentiment_analyzer = SentimentAnalyzer()


#BERT Embeder Class

In [ ]:

class BertEmbedder:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = TFBertModel.from_pretrained(model_name)

    def embed(self, text):
        input_ids = self.tokenizer.encode(text, truncation=True, padding=True, max_length=30)
        input_ids = tf.constant(input_ids)[None, :]
        outputs = self.model(input_ids)
        last_hidden_states = outputs[0]
        sentence_embedding = tf.reduce_mean(last_hidden_states, axis=1).numpy()
        return sentence_embedding

#Feature Extractor Class

In [ ]:

class FeatureExtractor:
    def __init__(self, preprocessor, ner, pos_tagger, sentiment_analyzer, bert_embedder):
        self.preprocessor = preprocessor
        self.ner = ner
        self.pos_tagger = pos_tagger
        self.sentiment_analyzer = sentiment_analyzer
        self.bert_embedder = bert_embedder

    def extract_features(self, text, label=None, label_value=None):
        # Preprocess the text
        tokens = self.preprocessor.preprocess(text)

        # Extract named entities
        named_entities = self.ner.recognize(text)

        # Extract POS tags
        pos_tags = self.pos_tagger.tag(text)

        # Analyze sentiment
        sentiment = self.sentiment_analyzer.analyze(text)

        # Get BERT embeddings
        bert_embedding = self.bert_embedder.embed(text)

        # Combine all the features
        features = {
            'pos_tags': pos_tags,
            'named_entities': named_entities,
            'sentiment': sentiment,
            'bert_embedding': bert_embedding,
            'Label': label,
            'Label_Value': label_value
        }

        return features

    def extract_features_from_df(self, df_list):
        train_features = []
        val_features = []
        test_features = []

        for i, df in enumerate(df_list):
            df_features = []

            for _, row in df.iterrows():
                text = row['Statement']
                label = row['Label']
                label_value = row['Label_Value']
                features = self.extract_features(text, label, label_value)
                df_features.append(features)

            if i == 0:
                train_features = df_features
            elif i == 1:
                val_features = df_features
            elif i == 2:
                test_features = df_features

        return train_features, val_features, test_features

In [ ]:

bert_fine_tuner = BertEmbedder()
feature_extractor = FeatureExtractor(preprocessor, NamedEntityRecognizer(), POSTagger(), sentiment_analyzer, bert_fine_tuner)

sentence = "Mr. President called the meeting at the White House"

features = feature_extractor.extract_features(sentence, label="True", label_value=1)
bert_embedding = features['bert_embedding']

print("Part of speech of input sentence:")
print(features['pos_tags'])
print("**********************************")
print("Named entities:")
for ent in features['named_entities'].ents:
    print(ent.text, ent.label_)

print("BERT sentence embedding:")
print(bert_embedding.shape)
print("*********************************")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Part of speech of input sentence:
[('Mr.', 'PROPN'), ('President', 'PROPN'), ('called', 'VERB'), ('the', 'DET'), ('meeting', 'NOUN'), ('at', 'ADP'), ('the', 'DET'), ('White', 'PROPN'), ('House', 'PROPN')]
**********************************
Named entities:
President PERSON
the White House FAC
BERT sentence embedding:
(1, 768)
*********************************


#Extracting Features from dataset

In [ ]:

# Extract features for the entire dataset
train_features, val_features, test_features = feature_extractor.extract_features_from_df(liar_df[:40])


#saving extracted features

In [ ]:

# Save the features
#with open("/content/drive/MyDrive/features/train_features.pkl", "wb") as f:
   # pickle.dump(train_features, f)

#with open("/content/drive/MyDrive/features/val_features.pkl", "wb") as f:
   # pickle.dump(val_features, f)

#with open("/content/drive/MyDrive/features/test_features.pkl", "wb") as f:
    #pickle.dump(test_features, f)

In [ ]:
pd.DataFrame(train_features).named_entities

,pos_tags,named_entities,sentiment,bert_embedding,Label,Label_Value
9297,"[(People, NOUN), (buy, VERB), (more, ADJ), (it...","(People, buy, more, items, during, Georgias, s...","(0.35, 0.35)","[[0.12604126, -0.22363362, 0.49643266, 0.26387...",True,1
